In [12]:
import numpy as np
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue

In [13]:
np.random.seed(42)

PROBLEM_SIZE = 5
NUM_SETS = 30
SETS = tuple(np.array([random() < .2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])

**Remarkable example(s)**\
Using these examples to test A* implementation

In [14]:
PROBLEM_SIZE = 6
NUM_SETS = 4
SETS = tuple(
    [np.array([True, True, True, False, False, False]),
    np.array([True, False, False, False, False, True]),
    np.array([False, True, False, False, True, False]),
    np.array([False, False, True, True, False, False])]
)
State = namedtuple('State', ['taken', 'not_taken'])

In [15]:
def goal_check(state):
    return np.all(reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))

def distance(state):
    return PROBLEM_SIZE - sum(reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))


In [16]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable."

In [17]:
def show_frontier(frontier: PriorityQueue):
    elements = list()

    print("\nFrontier: ")
    print('--------------------------------------')
    print('State\t\tDistance + Sets Taken')

    while not frontier.empty():
        priority, state = frontier.get()
        elements.append((priority, state))
        print(state.taken, '\t\t', priority,)
        
    print('--------------------------------------')


    for element in elements:
        frontier.put(element)

**Greedy Best First**\
Finds a good solution in a limited number of steps

In [18]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((distance(state), state))

counter = 0

_, current_state = frontier.get()

while not goal_check(current_state):
    counter += 1

    for action in current_state.not_taken:
        new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})
        frontier.put((distance(new_state), new_state))
    _, current_state = frontier.get()

print(f'Solved in {counter} steps')
print(f'Solution: {current_state.taken}')

Solved in 4 steps
Solution: {0, 1, 2, 3}


**Breadth First Search**\
Guarantees to find the best solution.

In [19]:
frontier = SimpleQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((distance(state), state))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state.not_taken:
        new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})
        frontier.put((distance(new_state), new_state))
    _, current_state = frontier.get()
    
print(f'Solved in {counter} steps')
print(f'Solution: {current_state.taken}')

Solved in 26 steps
Solution: {1, 2, 3}


**A***\
Complete and optimally efficient solution

Using as cost function the number of taken sets does not always guarantee to find the optimal solution. Use as example the *remarkable example* above. Set  <mark>verbose=True</mark> to understand how the priority queue behaves.

In [20]:
verbose = False

frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((distance(state), state))

counter = 0

_, current_state = frontier.get()

while not goal_check(current_state):
    counter += 1

    for action in current_state.not_taken:
        new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})
        frontier.put((distance(new_state) + len(current_state.taken), new_state))

    if verbose:
        show_frontier(frontier)
        print('\nThe current state is:', current_state)
        print('Distance:', distance(current_state))
        print('Tiles taken:', len(current_state.taken), '\n')
        
    _, current_state = frontier.get()

print(f'Solved in {counter} steps')
print(f'Solution: {current_state.taken}')

Solved in 11 steps
Solution: {0, 1, 2, 3}


Instead, using as cost function *(1.5 * number of taken sets)* guarantees to find the optimal solution.

In [21]:
verbose = False

frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((distance(state), state))

counter = 0

_, current_state = frontier.get()

while not goal_check(current_state):
    counter += 1

    for action in current_state.not_taken:
        new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})
        frontier.put((distance(new_state)+len(current_state.taken)*1.5, new_state))

    if verbose:
        show_frontier(frontier)
        print('\nThe current state is:', current_state)
        print('Distance:', distance(current_state))
        print('Sets taken:', len(current_state.taken), '\n')
        
    _, current_state = frontier.get()

print(f'Solved in {counter} steps')
print(f'Solution: {current_state.taken}')

Solved in 8 steps
Solution: {1, 2, 3}
